In [6]:
# %pip install aiohttp
# %pip install mysql-connector-python
%pip install nest_asyncio

Note: you may need to restart the kernel to use updated packages.


In [7]:
import asyncio
import aiohttp
import json
import sqlite3
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
import urllib
from pandas.io import sql
from datetime import datetime
import numpy as np
import requests
from time import perf_counter 
import nest_asyncio
nest_asyncio.apply()

# Semaphore code example

In [13]:
import asyncio
from random import randint


async def download(code):
    wait_time = randint(1, 3)
    print('downloading {} will take {} second(s)'.format(code, wait_time))
    await asyncio.sleep(wait_time)  # I/O, context will switch to main function
    print('downloaded {}'.format(code))


sem = asyncio.Semaphore(3)


async def safe_download(i):
    async with sem:  # semaphore limits num of simultaneous downloads
        return await download(i)


async def main():
    tasks = [
        asyncio.ensure_future(safe_download(i))  # creating task starts coroutine
        for i
        in range(9)
    ]
    await asyncio.gather(*tasks)  # await moment all downloads done


if __name__ ==  '__main__':
    loop = asyncio.get_event_loop()
    try:
        future = asyncio.ensure_future(main())
        loop.run_until_complete(future)
    finally:
        print("done")
#         loop.run_until_complete(loop.shutdown_asyncgens())
#         loop.close()

downloading 0 will take 1 second(s)
downloading 1 will take 2 second(s)
downloading 2 will take 2 second(s)
downloaded 0
downloading 3 will take 1 second(s)
downloaded 1
downloaded 2
downloading 4 will take 2 second(s)
downloading 5 will take 1 second(s)
downloaded 3
downloading 6 will take 3 second(s)
downloaded 5
downloading 7 will take 2 second(s)
downloaded 4
downloading 8 will take 3 second(s)
downloaded 6
downloaded 7
downloaded 8
done


# Performance Testing

In [14]:
class ConfigReader:
    def get_value(self, key):
        with open('config.json') as json_data_file:
            data = json.load(json_data_file)
        return data[key]

In [24]:
class DbManager:
    def __init__(self):
        reader = ConfigReader()
        db_connection = reader.get_value("db_connection")        
        self.conn_string = '{db_engine}{connector}://{user}:{password}@{server}/{database}'.format(
            db_engine=db_connection['db_engine'],
            connector=db_connection['connector'],
            user=db_connection['user'],
            password=db_connection['password'],
            server=db_connection['server'],
            database=db_connection['database'])

    def get_users(self):
        return sql.read_sql("SELECT distinct userId FROM ratings;", create_engine(self.conn_string))

## Get random users

In [25]:
n_rand_users = 10
dbManager = DbManager()
db_users = dbManager.get_users()
n_rand_users = db_users.sample(n=n_rand_users)

## Test recommendation endpoint

In [26]:
base_url = 'http://127.0.0.1:8000'
algo_rec = 'popular'
algo_pred = 'biasedmf' # 'biasedmf'
n_recs = 5
items = "10,20,30,40,50"

### Parallel performance with semaphore

### Parallel performance

In [27]:
def print_stats(time_taken_all, num_requests):
    print(f'Total response time: {time_taken_all}')
    print(f'Throughput (requests per second): {num_requests / time_taken_all}')
    
async def get_recs():
    tasks = []    
    num_requests = len(n_rand_users)
    print(f'Number of requests: {num_requests}')
    start_recs = perf_counter()

    async with aiohttp.ClientSession() as session:
        for idx, row in n_rand_users.iterrows():
            task = asyncio.ensure_future(get_user_results(row['userId'], n_recs, algo_rec, None, session))
            tasks.append(task)     

        responses = await asyncio.gather(*tasks)
        time_taken_all = perf_counter() - start_recs
        print_stats(time_taken_all, num_requests)

async def get_preds():
    tasks = []    
    num_requests = len(n_rand_users)
    print(f'Number of requests: {num_requests}')
    start_preds = perf_counter()

    async with aiohttp.ClientSession() as session:
        for idx, row in n_rand_users.iterrows():
            task = asyncio.ensure_future(get_user_results(row['userId'], None, algo_pred, items, session))
            tasks.append(task)         

        responses = await asyncio.gather(*tasks)
        time_taken_all = perf_counter() - start_preds
        print_stats(time_taken_all, num_requests)

async def get_user_results(userId, nr_recs, algo, items, session):
    is_a_rec_request = True if algo == 'popular' or algo == 'topn' else False
    if is_a_rec_request:
        url = f'{base_url}/algorithms/{algo}/recommendations?user_id={userId}&num_recs={nr_recs}'
    else:
        url = f'{base_url}/algorithms/{algo}/predictions?user_id={userId}&items={items}'
    async with session.get(url) as resp:
        data = await resp.json()
        #data = resp.json()

### Single thread performance

In [28]:
def print_stats_single_thread(times, time_taken_all, num_requests):
    print(f'Total response time: {time_taken_all}')
    print(f'Throughput (requests per second): {num_requests / time_taken_all}')
    
def get_recs_single_thread():
    times = []
    num_requests = len(n_rand_users)
    print(f'Number of requests: {num_requests}')
    start_recs = perf_counter()
    for idx, row in n_rand_users.iterrows():
        start = perf_counter()
        get_user_results_single_thread(row['userId'], n_recs, algo_rec, None)
        time_taken = perf_counter() - start
        times.append(time_taken)
#        print(f'Response time: {time_taken}')
    time_taken_all = perf_counter() - start_recs
    print_stats_single_thread(times, time_taken_all, num_requests)

def get_preds_single_thread():
    times = []
    start_preds = perf_counter()
    num_requests = len(n_rand_users)
    print(f'Number of requests: {num_requests}')    
    for idx, row in n_rand_users.iterrows():
        start = perf_counter()
        get_user_results_single_thread(row['userId'], None, algo_pred, items)
        time_taken = perf_counter() - start
        times.append(time_taken)
#        print(f'Response time: {time_taken}')
    time_taken_all = perf_counter() - start_preds
    print_stats_single_thread(times, time_taken_all, num_requests)
    
def get_user_results_single_thread(userId, nr_recs, algo, items):
    is_a_rec_request = True if algo == 'popular' or algo == 'topn' else False
    if is_a_rec_request:
        url = f'{base_url}/algorithms/{algo}/recommendations?user_id={userId}&num_recs={nr_recs}'
    else:
        url = f'{base_url}/algorithms/{algo}/predictions?user_id={userId}&items={items}'

    r = requests.get(url)
    data = r.json()

In [31]:
print('-------------------------------------')
print('Predictions results')
loop = asyncio.get_event_loop()
future = asyncio.ensure_future(get_preds())
loop.run_until_complete(future)

print('-------------------------------------')
print('Predictions results in single thread')
get_preds_single_thread()

-------------------------------------
Predictions results
Number of requests: 10
Total response time: 0.38597445599998537
Throughput (requests per second): 25.9084502731973
-------------------------------------
Predictions results in single thread
Number of requests: 10
Total response time: 0.09544832000005954
Throughput (requests per second): 104.76873767913109


In [30]:
# print('-------------------------------------')
# print('Recomendation results')
# loop = asyncio.get_event_loop()
# loop.run_until_complete(get_recs())
# loop.close()

# print('-------------------------------------')
# print('Recommendations results in single thread')
# get_recs_single_thread()